In [22]:
# input
n = 4
c = (2,2,3,2)       # the number of kWh of electricity you consumed this day
p = (2,5,8,10)      # the variable price per kWh, what you would pay this day if you have no fixed plan.
q = (4,3,4,4)       # the price per kWh for a 1-year plan started this day.
r = (10,9,8,7)      # the price per kWh for a 2-year plan started this day. 
s = (10,10,10,10)   # the price per kWh for a 5-year plan started this day.

In [25]:
import numpy as np
def p2(n):
    dp = np.zeros(n)
    sum_c_365 = np.empty(n)

    for i in range(n):
        sum_c_365[i] = sum(c[i:i+365])
    
    for i in range(n):
        dp[i] = min()

    
    return sum_c_365



In [26]:
p2(4)

array([9., 7., 5., 2.])

In [3]:
def P(x):
    global n, memo
    n = x
    memo = {}
    return p1(0)

def p1(d):
    if d > n:
        return 10**100
    elif d == n:
        return 0

    if d in memo:
        return memo[d]

    q1 = 365 if n-d > 365 else n-d
    r1 = 730 if n-d > 730 else n-d
    s1 = 1825 if n-d > 1825 else n-d

    result = min(c[d]*p[d] + p1(d+1), sum(c[d:d+q1])*q[d] + p1(d+q1), sum(c[d:d+r1])*r[d] + p1(d+r1), sum(c[d:d+s1])*s[d] + p1(d+s1))

    memo[d] = result
    return result


In [4]:
P(4)

25